# Fonctions

## nan_rate

In [ ]:
nan_rate = []
for col in df.columns:
   #print(round(df[col].isna().sum()/df.shape[0]*100, 2), "%")
    nan_rate.append([col,round(df[col].isna().sum()/df.shape[0]*100, 1) ])
print(pd.DataFrame(nan_rate))


## dict

In [ ]:
# Création du dictionnaire clé = station, valeurs = grandeurs non mesurées
dict = {}
for i in range(df['Location'].unique().shape[0]):  # df['Location'].unique().shape[0]: le nombre de stations
    measures = []
    non_measures = []
    station_df = df.loc[df['Location'] == df['Location'].unique()[i]] #création d'un df pour une station
    for col in station_df.columns:
        if station_df[col].isna().sum()/station_df.shape[0] != 1: # si le pourcentage de nan est différent de 1
            measures.append(col) # on ajoute la grandeur mesurée à 'measures
        else:
            non_measures.append(col) # sinon, on l'ajoute à 'non_measures'
    if non_measures == []:
        dict[df['Location'].unique()[i]] = ['all']
    else:
        dict[df['Location'].unique()[i]] = non_measures

## modalites

In [ ]:
### Liste des différentes modalités de non-mesures:
modalites = []
for station in dict:
    if dict[station] not in modalites:
        modalites.append(dict[station])


## dict_reverse

In [ ]:
### Création du dictionnaire inverse: clé = liste concaténée des grandeurs non mesurées, valeurs = liste des stations correspondantes
dict_reverse = {}
for i, mod in zip(range(len(modalites)), modalites):
    liste_stations = []
    for station in dict:
        if dict[station] == mod:
            liste_stations.append(station)
    dict_reverse["_".join(modalites[i])] = liste_stations # J'ai appris qu'un dictionnaire ne peut pas prendre de liste en guise de clé. 
                                                          #J'ai décidé de concaténer le nom des colonnes non mesurées avec un "_"       
    #print(mod, ":", liste_stations) 

## Heatmap des nan

In [ ]:
def nan_rate_count(col):
    nr = round(col.isna().sum()/col.shape[0]*100, 2)
    return nr

nan_map = df.groupby('Location').agg(nan_rate_count)

plt.figure(figsize = (20,10))
ax = sns.heatmap(nan_map.T, annot = False, cmap = "RdYlGn_r", square = True, linewidth = 1, annot_kws={"size":6}, fmt=".0f", cbar_kws = {"label": "Percentage"})

## Mise en forme
plt.title("Taux de NaN", fontweight = "bold", fontsize = 16)
plt.xlabel("Variable", fontweight = "bold")
plt.ylabel("Location", fontweight = "bold")
plt.xticks(rotation = 70)
cbar_axes = ax.figure.axes[-1].yaxis.label.set_weight("bold")
plt.show()

# choix de la palette: voir ici :
# https://matplotlib.org/stable/users/explain/colors/colormaps.html
# on ajoute "_r" à la fin de la palette pour l'inverser.

## rapport_jours_manqués

In [ ]:
def rapport_jours_manqués(station):
    print("Pour la station", station, ":")
    lieu = df.loc[df['Location'] == station]
    print("Première date enregistrée:", lieu['Date'].dt.date.min())
    print("Dernière date enregistrée:", lieu['Date'].dt.date.max())
    print("Nombre de dates mesurées:", len(lieu['Date']))
    delta_t = lieu['Date'].dt.date.max()-lieu['Date'].dt.date.min()
    print("Nombre total de jours entre le début et la fin:", delta_t.days)
    print("Il manque donc", delta_t.days-len(lieu['Date']), "jours pour la station", station, ".")
    print("-----------------------------------------------------------------------")


## données_dates

In [ ]:
def données_dates(station):
    lieu = df.loc[df['Location'] == station]
    date_min = lieu['Date'].dt.date.min()
    date_max = lieu['Date'].dt.date.max()
    n_mes = len(lieu['Date'])
    period = date_max - date_min
    n_missed = period.days - n_mes
    print("Il manque", n_missed, "jours pour la station", station, ".") # j'aoute le message pour avoir un aperçu global.

## dict_dates

In [ ]:
dict_dates = {"endroit" : [],
             "date_debut" : [], 
             "date_fin" : [],
             "n_jours_tot": [],
             "n_jours_mes": [],
             "n_jours_miss": []}
keys = ["endroit", "date_debut", "date_fin", "n_jours_tot", "n_jours_mes","n_jours_miss"]

for station in df['Location'].unique():
    values = []
    lieu = df.loc[df['Location'] == station]
    date_min = lieu['Date'].dt.date.min()
    date_max = lieu['Date'].dt.date.max()
    n_mes = len(lieu['Date'])
    period = date_max - date_min
    n_missed = period.days - n_mes
    values = [str(station), date_min, date_max, period, n_mes, n_missed]
    
    for j, key in enumerate(keys):
        dict_dates[key].append(values[j])
df_dates = pd.DataFrame.from_dict(dict_dates)
df_dates.head(49)

## rapport_stat

In [ ]:
def rapport_stat(mesure):
    df_mes = df[mesure].copy()
    df_mes.dropna(inplace = True)

    ext_min = df_mes.quantile(q = 0.25)-1.5*(df_mes.quantile(q = 0.75)-df_mes.quantile(q = 0.25))
    # ext_min est la frontière pour les valeurs extrèmes minimales, cad < Q1 -1.5*IQR
    ext_max = df_mes.quantile(q = 0.75)+1.5*(df_mes.quantile(q = 0.75)-df_mes.quantile(q = 0.25))
    # ext_max est la frontière pour les valeurs extrèmes maximales, cad > Q + 1.5*IQR

    df_mes_min = df_mes.loc[df_mes < ext_min]
    print("Il y a ", df_mes_min.shape[0], "valeurs extrèmes inférieures pour la mesure", str(mesure),  "sur", df_mes.shape[0], "valeurs mesurées.")
    p_min = round(df_mes_min.shape[0]/df_mes.shape[0]*100, 2)
    print("Cela correspond à",p_min , "% des valeurs de cette colonne.")
    print("-------------------------------------------")

    df_mes_max = df_mes.loc[df_mes > ext_max]
    print("Il y a ", df_mes_max.shape[0], "valeurs extrèmes supérieures pour la mesure" , str(mesure),  "sur", df_mes.shape[0], "valeurs mesurées.") # OC : remplacé « inférieures » par « supérieures »
    p_max = round(df_mes_max.shape[0]/h3.shape[0]*100, 2)
    print("Cela correspond à",p_max , "% des valeurs de cette colonne.")
    
    import statsmodels.api as sm
    plt.figure(figsize=(7,5))

    plt.boxplot(df_mes)
    plt.xlabel(str(mesure))
    plt.ylabel('Valeurs en mm')
    plt.title('Distribution de la grandeur '+ mesure)
    plt.axes([0.65, 0.65, 0.2, 0.15])
    plt.hist(df_mes, rwidth = 0.8, color = 'royalblue')
    plt.title('Histogramme')
    #plt.xticks([])
    plt.yticks([])

    #sm.qqplot(df_mes, fit = True, line = '45')
    plt.show()

## distrib_grandeur_location

In [ ]:
def distrib_grandeur_location(mesure, unit, color_box, color_mark):
    # Initialisation de l'existence de stations météo ayant zéro entrée dans la colonne « mesure »

    df_location_mes_counts = df[["Location", mesure]].groupby("Location").count()

    ## Initialisation de la toile
    fig, ax = plt.subplots(figsize = (12, 8))

    ## Traçage
    sns.boxplot(x = df["Location"].sort_values(ascending = True), 
            y = df[mesure], 
            color = color_box, 
            flierprops={"marker": "x", 
                        "markeredgecolor": color_mark})

    ## Mise en forme
    plt.xticks(rotation = 90)
    plt.ylim(df[mesure].min(), df[mesure].max())
    plt.ylabel(mesure +  " en "+ unit)
    plt.title("Distribution de " + mesure + " en fonction des stations", fontweight = "bold");

    ### Mise en évidence des stations météo ne disposant pas de données pour « mesure »
    for index in range(len(df_location_mes_counts)):
        if df_location_mes_counts.iloc[index][0] == 0:
            ax.get_xticklabels()[index].set_color("#dda15e")
            ax.get_xticklabels()[index].set_weight("bold")

 # AW: La fonction renvoie bien un graphique. 
 # J'ai un problème non résolu: dans l'écriture de la fonction, j'appelle mesure (entre parenthèses)
 # Pour qu'elle fonctionne, il faut rentrer 'Pressure9am', au lieu de simplement Pressure9am.
 # Est-ce possible de modifier ça?

## Création des 9 datasets 

In [ ]:
# Ce bloc permet de créer les 9 datasets, un par modalité.
# il utilise dict, dict_reverse, modalites.

dict_reverse_keys = list(dict_reverse.keys()) # Création d'une liste contenant les modalités concaténées.

for key in dict_reverse_keys:
    obj = []
    for i, station in enumerate(dict_reverse[key]):
        stat = df.loc[df['Location'] == station]
        obj.append(stat)
    
    concat = pd.concat(obj, axis = 0) 

    if key != 'all':
        concat_drop = concat.drop(columns = str.split(key, sep = "_"))
    else: concat_drop = concat  # ligne ajoutée pour régler le problème de la création du df 'all'

    title = str(key) + ".csv"
    concat_drop.to_csv(title)